In [ ]:
import os
import pickle
import numpy as np
import SimpleITK as sitk

### Generating probability map
The code below generates a NIfTI probability map from the files outputted from the nnU-Net after inference.
The probability map from the nnU-Net is outputted as a numpy array (.npz) and a corresponding pickle file (.pkl) that contains the information to reconstruct a NIfTI image.

In the code below all images in the test set is converted and used as an example. To convert the other datasets only the paths has to change.

In [2]:
#Adding all subjects from the chosen set into a list
all_files_list = []
for path, dirs, files in os.walk("/home/maarhus/Documents/Master/data_reoriented/MS_DATA_TEST"): 
    if len(dirs) == 0:
        all_files_list.append(os.path.basename(path))

In [197]:
#Iterating through all subjects in the test set
for file in all_files_list:
    #Path to pickle file
    pkl_path = f'/home/maarhus/Documents/Master/nnUNET_data/directory_env/nnUnetFrame/dataset/nnUnet_raw/\
                    nnUnet_raw_data/nnUNet_raw_data/Task555_MultipleSclerosis/preds/MS_{file}.pkl'

    #Open pkl file to find affinities
    with open(pkl_path, 'rb') as f:
        data = pickle.load(f)

    #Path to numpy array containing softmax probabilities
    npz_path = f'/home/maarhus/Documents/Master/nnUNET_data/directory_env/nnUnetFrame/dataset/nnUnet_raw/\
                    nnUnet_raw_data/nnUNet_raw_data/Task555_MultipleSclerosis/preds/MS_{file}.npz'
    
    #Loading into an array
    npz_array = np.load(npz_path)

    #Get probabilities from the numpy array
    prob_img = npz_array['softmax'][1,:,:,:]
    prob_img = np.array(prob_img,dtype='float64')

    #Remove noise from the image, values under 0.1 or over 1 gets set to 0
    prob_img[prob_img < 0.1] = 0
    prob_img[prob_img > 1] = 0

    #Save probabilities as NIfTY image using affinities from the pickle file
    seg_resized_itk = sitk.GetImageFromArray(prob_img)
    seg_resized_itk.SetSpacing(data['itk_spacing'])
    seg_resized_itk.SetOrigin(data['itk_origin'])
    seg_resized_itk.SetDirection(data['itk_direction'])
    sitk.WriteImage(seg_resized_itk, f"/home/maarhus/Documents/Master/nnUNET_data/directory_env/nnUnetFrame/dataset/nnUnet_raw/\
                                            nnUnet_raw_data/nnUNet_raw_data/Task555_MultipleSclerosis/preds/\
                                                probmap_folder/probmap_{file}.nii")